# Imports

In [146]:
import pandas as pd
import utm

# Funcrions to prepare a file from 1 year

In [157]:
# Translating coordinates from UTM32 to long lat 
def translate(x,y):
    x_tr,y_tr = utm.to_latlon(x, y, 32, 'U')
    return x_tr,y_tr

# Process 1 file 
def process_file(file_name,vehicle_type):
    # Load the file
    df_tmp = pd.read_excel(file_name, skiprows=10)  
    
    # Select columns
    df_tmp = df_tmp[['(UTM32)', '(UTM32).1',  'Dato', 'kl.00-01',
           'kl.01-02', 'kl.02-03', 'kl.03-04', 'kl.04-05', 'kl.05-06', 'kl.06-07',
           'kl.07-08', 'kl.08-09', 'kl.09-10', 'kl.10-11', 'kl.11-12', 'kl.12-13',
           'kl.13-14', 'kl.14-15', 'kl.15-16', 'kl.16-17', 'kl.17-18', 'kl.18-19',
           'kl.19-20', 'kl.20-21', 'kl.21-22', 'kl.22-23', 'kl.23-24']]

    # Rename columns
    df_tmp.columns = ['(UTM32)','(UTM32).1','time','0',
                          '1','2','3','4','5','6','7', '8',
                          '9','10','11','12','13','14','15',
                          '16','17','18','19','20','21','22','23']

    # Translate coordinates from utm to longtitude and lattitude
    df_tmp['point'] = df_tmp.apply(lambda x: translate(x['(UTM32)'], x['(UTM32).1']), axis=1)
    df_tmp['latitude'] = df_tmp.point.apply(lambda x: x[0])
    df_tmp['longitude'] = df_tmp.point.apply(lambda x: x[1])

    # Drop temporary colmns
    df_tmp = df_tmp[['latitude','longitude','time','0',
                     '1','2','3','4','5','6','7', '8',
                     '9','10','11','12','13','14','15',
                     '16','17','18','19','20','21','22','23']]

    # Change date into datetime
    df_tmp['time'] =  pd.to_datetime(df_tmp['time'])

    # Rotate hours
    df_tmp = df_tmp.melt(id_vars = ['latitude','longitude','time'],
                         var_name = 'hour', 
                         value_name = "count").sort_values('time')
    print(df_tmp.shape)
    return df_tmp

# Combine and save datasets

In [ ]:
# Concatinate processed bicycle files into 1 dataframe and save it in csv
transport = 'bicycle'
df = process_file('Material/cykeltaellinger-2010.xlsx', transport)
for year in range(2011,2015):
    name_of_file = 'Material/cykeltaellinger-{}.xlsx'.format(year)
    df = pd.concat([df,process_file(name_of_file, transport)], axis=0)
df.to_csv('Material/copenhagen_{}_counts.csv'.format(transport))

In [ ]:
df

In [ ]:
# Concatinate processed vehicle files into 1 dataframe and save it in csv
transport = 'vehicle'
df = process_file('Material/cykeltaellinger-2010.xlsx', transport)
for year in range(2011,2015):
    name_of_file = 'Material/cykeltaellinger-{}.xlsx'.format(year)
    df = pd.concat([df,process_file(name_of_file, transport)], axis=0)
df.to_csv('Material/copenhagen_{}_counts.csv'.format(transport))

In [ ]:
df